In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd '/content/gdrive/MyDrive/Colab Notebooks'
%ls

In [ ]:
!unzip 'flowers-recognition.zip'

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, Activation
from keras import models
from keras import layers
import tensorflow as tf
import os
import os.path
from pathlib import Path
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from keras.optimizers import RMSprop,Adam
import glob
from PIL import Image
from sklearn.preprocessing import StandardScaler
from keras.preprocessing import image
from keras.layers import SimpleRNN
from keras.layers import LSTM

In [ ]:
Flowers_Path = Path('/content/gdrive/MyDrive/Colab Notebooks/flowers')
Flowers_Path = list(Flowers_Path.glob(r"*/*.jpg"))
Flowers_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Flowers_Path))

In [ ]:
Flowers_Path_Series = pd.Series(Flowers_Path,name="PATH").astype(str)
Flowers_Labels_Series = pd.Series(Flowers_Labels,name="CATEGORY")

Flowers_Data= pd.concat([Flowers_Path_Series,Flowers_Labels_Series],axis=1)
Flowers_Data = Flowers_Data.sample(frac=1).reset_index(drop=True)


In [ ]:
plt.figure(figsize=(8,8))
Flowers_percent=Flowers_Data.groupby('CATEGORY').PATH.count()/len(Flowers_Data)

print(Flowers_percent)

Flowers_percent.plot.bar()
plt.title("Percent of images in each class")
plt.ylabel('Percent(%)')
plt.title('Number of images in each class')

plt.savefig('Flower percents.png')



#I/ Data DataGenerator:

In [ ]:
Train_Generator = ImageDataGenerator(rescale=1./255,
                                    zoom_range=0.2,
                                    rotation_range=40,
                                    shear_range=0.2,
                                     channel_shift_range=0.2,
                                     fill_mode="nearest",
                                    horizontal_flip=True)

Test_Generator = ImageDataGenerator(rescale=1./255)

In [ ]:
Train_Data,Test_Data = train_test_split(Flowers_Data,train_size=0.8,random_state=123,shuffle=True)
Test_Data, Val_Data  = train_test_split(Test_Data,train_size=0.5,random_state=123,shuffle=True)

In [ ]:
IMG_SIZE = (224, 224)
INP_SHAPE = (*IMG_SIZE, 3)
BATCH_SIZE = 32

images_train = Train_Generator.flow_from_dataframe(dataframe=Train_Data,
                                                   x_col="PATH",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=32,
                                                   target_size=IMG_SIZE)

images_val = Test_Generator.flow_from_dataframe(dataframe=Val_Data,
                                                   x_col="PATH",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=32,
                                                   target_size=IMG_SIZE)

images_test = Test_Generator.flow_from_dataframe(dataframe=Test_Data,
                                                   x_col="PATH",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=32,
                                                   target_size=IMG_SIZE)


#II/ Training model:

##1/ CNN:

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = INP_SHAPE))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation = "softmax"))

model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
checkpoint_path ="CNN.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights every 10 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=False,
    save_freq="epoch",
    save_best_only=True)

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(images_train, validation_data=images_val,batch_size = BATCH_SIZE,epochs=30,callbacks=[cp_callback])

##2.MobileNet

In [ ]:
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# MobileNet
model=Sequential()
model.add(MobileNet(input_shape=(224, 224, 3), include_top = False,dropout=0.01, weights = 'imagenet'))
model.add(Flatten())
model.add(Dense(64, input_dim=64,
                kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l2(0.01)))

model.add(Dropout(0.1))
model.add(Dense(5, activation="softmax"))
'''
model = Sequential([base_network, 
                    flat,
                    den])
'''
model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=["accuracy"])
model.summary()

In [ ]:
# Frozen EfficientNet network
for layer in model.layers[:1]:
  layer.trainable = False

for layer in model.layers:
  print('Layer: {} ; Trainable: {}'.format(layer, layer.trainable))

In [ ]:
# Include the epoch in the file name (uses `str.format`)
#checkpoint_path = "cp-{epoch:04d}-{val_accuracy}.ckpt"
checkpoint_path ="MobileNet.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
# Create a callback that saves the model's weights every 10 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=False,
    save_freq="epoch",
    save_best_only=True)

# Save the weights using the `checkpoint_path` format
'''
period=N:int: Save weights, every N-epochs.
save_freq="epoch"||N:int: Save weights, each epoch || every N-batches. 
save_best_only=True: Save weights of only best model.

'''
# Train the model with the new callback
model.fit(images_train, validation_data=images_val,batch_size = BATCH_SIZE,epochs=20,callbacks=[cp_callback])

In [ ]:
# Defrost EfficientNet network
#model=load_model('MobileNet_ver2.h5')
for layer in model.layers[:1]:
  layer.trainable = True

for layer in model.layers:
  print('Layer: {} ; Trainable: {}'.format(layer, layer.trainable))


In [ ]:
model.fit(images_train, validation_data=images_val,epochs=10,callbacks=[cp_callback])

# III/ Evaluating Model

In [ ]:
model=load_model('MobileNet.h5')
model.evaluate(images_test)

In [ ]:
model=load_model('CNN_ver2.h5')
model.evaluate(images_test)